In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [1]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...


2025-01-21 16:05:21.562572: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# === STEP 0: Imports: Complete


In [2]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_000.yaml'
yaml_path1 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_001.yaml'
yaml_paths = [yaml_path0, yaml_path1]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


Load Train, Val, and Test Datasets into Memory

In [3]:
import pickle
import os

# File path to load train, test, and val datasets
train_test_val_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v01"

train_dataset_path = train_test_val_dataset_path + "/train.pkl"
val_dataset_path = train_test_val_dataset_path + "/val.pkl"
# test_dataset_path = train_test_val_dataset_path + "/test.pkl"

with open(train_dataset_path, "rb") as f:
    train_dataset = pickle.load(f)
    print("Train dataset leaded to memory")
with open(val_dataset_path, "rb") as f:
    validation_dataset = pickle.load(f)
    print("Val dataset leaded to memory")
# with open(test_dataset_path, "rb") as f:
#     test_dataset = pickle.load(f)
#     print("Test dataset leaded to memory")

Train dataset leaded to memory
Val dataset leaded to memory


Create and Train Models

In [4]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.create_models([yaml_path0])

# === STEP 4: MODEL MANAGER: Starting ...


In [5]:
train_dataset.examples[0].features
len(train_dataset.examples[0].features) # should be 178

178

In [6]:
# If no val dataset, code requires to push None instead
val_dataset = validation_dataset if validation_dataset is not None else None

model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)
print("# === STEP 4: MODEL MANAGER: Complete")

INFO:root:Training model 'f40d19f2194bf44123fef3e3c7d48f0e' for 20 epochs with batch size 32.


Epoch 1/20
557/557 [==============================] - 3s 4ms/step - loss: 3463.3567 - accuracy: 0.5346 - val_loss: 6286.8125 - val_accuracy: 0.3985
Epoch 2/20
557/557 [==============================] - 2s 4ms/step - loss: 711.9361 - accuracy: 0.5454 - val_loss: 4486.9121 - val_accuracy: 0.3953
Epoch 3/20
557/557 [==============================] - 2s 4ms/step - loss: 428.3598 - accuracy: 0.5314 - val_loss: 609.3102 - val_accuracy: 0.3953
Epoch 4/20
557/557 [==============================] - 3s 5ms/step - loss: 238.9464 - accuracy: 0.5497 - val_loss: 188.7646 - val_accuracy: 0.6047
Epoch 5/20
557/557 [==============================] - 3s 5ms/step - loss: 221.5906 - accuracy: 0.5559 - val_loss: 184.5617 - val_accuracy: 0.6047
Epoch 6/20
557/557 [==============================] - 2s 4ms/step - loss: 215.3961 - accuracy: 0.5537 - val_loss: 177.7046 - val_accuracy: 0.6047
Epoch 7/20
557/557 [==============================] - 2s 3ms/step - loss: 207.0026 - accuracy: 0.5566 - val_loss: 174.522

INFO:root:Model 'f40d19f2194bf44123fef3e3c7d48f0e': Finished training.


Model saved successfully at: models/f40d19f2194bf44123fef3e3c7d48f0e/model_weights_f40d19f2194bf44123fef3e3c7d48f0e.pth
# === STEP 4: MODEL MANAGER: Complete


In [7]:
predictions = []
for model in models:
    print(model)
    predictions.append(model.predict(validation_dataset.examples, return_target_labels=True))

In [10]:
predictions[0]

,final_score_A,final_score_B,target_final_score_A,target_final_score_B
0,104.633408,101.997154,117.0,102.0
1,104.590019,101.963638,85.0,95.0
2,100.301682,97.563698,87.0,80.0
3,101.331535,98.601669,106.0,82.0
4,102.652214,99.951218,101.0,75.0
...,...,...,...,...
3812,102.171257,99.438766,116.0,123.0
3813,101.948509,99.212097,125.0,127.0
3814,100.462669,97.727226,85.0,96.0
3815,97.721458,94.996109,90.0,92.0


In [12]:
total_examples = 0
correct_predictions = 0

for row in predictions[0].itertuples(index=False):
    total_examples += 1

    # Compare predictions and target labels for the current feature
    if (row.final_score_A > row.final_score_B and row.target_final_score_A > row.target_final_score_B) or \
       (row.final_score_A < row.final_score_B and row.target_final_score_A < row.target_final_score_B):
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_examples
print(f"Accuracy across all features on validation dataset: {accuracy}")

Accuracy across all features on validation dataset: 0.6046633481791983


In [13]:
%tensorboard --logdir logs/fit/f40d19f2194bf44123fef3e3c7d48f0e

UsageError: Line magic function `%tensorboard` not found.
